In [1]:
import pandas as pd
import numpy as np

In [41]:
raw_data = pd.read_csv('../data/raw/df_merged11.csv')
no_hospice_raw_data = raw_data[raw_data['discharge_location'] != 'HOSPICE']

print(len(no_hospice_raw_data), len(raw_data))
print(len(no_hospice_raw_data[no_hospice_raw_data['deathtime'].isna()]), len(
    no_hospice_raw_data[~no_hospice_raw_data['deathtime'].isna()]))

989 1033
857 132


In [30]:
columns = no_hospice_raw_data.columns

In [34]:
# help me look at null values in each column
for column in columns:
    print(column, no_hospice_raw_data[column].isnull().sum())

subject_id 0
hadm_id 0
gender 0
anchor_age 0
race 0
admittime 0
dischtime_x 0
hospital_expire_flag 0
admission_type 0
insurance 0
marital_status 90
stay_id 449
intime 449
outtime 449
los 449
deathtime 857
discharge_location 116
item_51006 49
item_51221 58
item_51222 58
item_51265 60
item_51301 59
item_51237 145
item_51274 145
item_51275 148
item_50863 305
item_50878 293
item_50885 332
total_drugs 13
unique_drugs 13
lods_score 449
oasis_score 449
procedure_221214 301
procedure_224275 301
procedure_224277 301
procedure_225402 301
procedure_225459 301
procedure_225792 301
procedure_229351 301
procedure_229581 301
ph 616
PaO2 616
PaCO2 616
sodium 63
potassium 68
calcium 238
creatinine 49
bun 41
urine_output_total 499
pt 132
inr 132
ptt 137
systolic_bp 450
diastolic_bp 450
mean_arterial_pressure 450
heart_rate 449
resp_rate 449
temperature 455
spo2 449
glucose 468
starttime 642
endtime 642
hospital_stay_duration 0
icu_stay_duration 449
ventilation_duration 642


- id: subject_id, hadm_id, 

- one hot encode: gender, race, admission_type, insurance, martial status

- delete: admittime, dischtime_x, starttime, endtime, hospital_expire_flag,  (50863, 50878, 508885 or also 51237 51274 51275) , intime, outtime

- change to bool of null or not null: ventilation_duration; stay_id (turn into 'went into ICU or not'); procedure (probably can only pick 1 since all 301)

- fillna: total_drugs, unique_drugs (?)

    - los, icu time (??)

    - write on discussion: we choose to record certain features; this induces biases.

- dropna: labtest
    - cannot do mean median impute, cannot do null-fill: not missing at random. (write on report)


In [85]:
drop_columns = ['deathtime', 'subject_id', 'hadm_id', 'admittime', 'dischtime_x', 'starttime', 'endtime', 'intime', 'outtime', 'hospital_expire_flag',
                'item_50863', 'item_50878', 'item_50885', 'procedure_224275', 'procedure_224277', 'procedure_225402', 'procedure_225459', 'procedure_225792',
                'procedure_229351', 'procedure_229581', 'icu_stay_duration', 'los', 'discharge_location']

# drop_columns.extend(['item_51237', 'item_51274', 'item_51275'])

fillna_columns = ['total_drugs', 'unique_drugs', 'marital_status']

booleanise_columns = ['ventilation_duration', 'stay_id', 'procedure_221214']

In [86]:
drop_columns_df = no_hospice_raw_data.drop(columns=drop_columns)

for column in fillna_columns:
    drop_columns_df[column].fillna(0, inplace=True)

for column in booleanise_columns:
    drop_columns_df[column] = drop_columns_df[column].apply(
        lambda x: 1 if not pd.isna(x) else 0)

/var/folders/xg/8w_3dndd6l5c3n99vd7vd3f40000gn/T/ipykernel_33998/797526771.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  drop_columns_df[column].fillna(0, inplace=True)


In [87]:
drop_columns_df.columns

Index(['gender', 'anchor_age', 'race', 'admission_type', 'insurance',
       'marital_status', 'stay_id', 'item_51006', 'item_51221', 'item_51222',
       'item_51265', 'item_51301', 'item_51237', 'item_51274', 'item_51275',
       'total_drugs', 'unique_drugs', 'lods_score', 'oasis_score',
       'procedure_221214', 'ph', 'PaO2', 'PaCO2', 'sodium', 'potassium',
       'calcium', 'creatinine', 'bun', 'urine_output_total', 'pt', 'inr',
       'ptt', 'systolic_bp', 'diastolic_bp', 'mean_arterial_pressure',
       'heart_rate', 'resp_rate', 'temperature', 'spo2', 'glucose',
       'hospital_stay_duration', 'ventilation_duration'],
      dtype='object')

In [88]:
len(drop_columns_df.dropna())

229

In [89]:
# help me look at null values in each column
second_drop_columns = []

for column in drop_columns_df.columns:
    print(column, drop_columns_df[column].isnull().sum())
    if drop_columns_df[column].isnull().sum() >= 449:
        second_drop_columns.append(column)

gender 0
anchor_age 0
race 0
admission_type 0
insurance 0
marital_status 0
stay_id 0
item_51006 49
item_51221 58
item_51222 58
item_51265 60
item_51301 59
item_51237 145
item_51274 145
item_51275 148
total_drugs 0
unique_drugs 0
lods_score 449
oasis_score 449
procedure_221214 0
ph 616
PaO2 616
PaCO2 616
sodium 63
potassium 68
calcium 238
creatinine 49
bun 41
urine_output_total 499
pt 132
inr 132
ptt 137
systolic_bp 450
diastolic_bp 450
mean_arterial_pressure 450
heart_rate 449
resp_rate 449
temperature 455
spo2 449
glucose 468
hospital_stay_duration 0
ventilation_duration 0


In [90]:
second_drop_columns_df = drop_columns_df.drop(columns=second_drop_columns)

In [91]:
second_drop_columns_df.dropna()

,gender,anchor_age,race,admission_type,insurance,marital_status,stay_id,item_51006,item_51221,item_51222,...,sodium,potassium,calcium,creatinine,bun,pt,inr,ptt,hospital_stay_duration,ventilation_duration
0,M,40,BLACK/AFRICAN AMERICAN,EW EMER.,Other,SINGLE,1,80.0,17.9,5.5,...,137.0,5.3,9.2,12.6,80.0,11.0,1.0,26.4,5.765972,1
1,M,79,WHITE,URGENT,Medicare,MARRIED,0,11.0,41.5,13.1,...,144.0,4.4,8.5,0.7,11.0,12.8,1.2,24.1,5.393056,0
3,M,73,WHITE,URGENT,Medicare,MARRIED,1,21.0,42.5,14.3,...,142.0,4.6,8.9,0.7,21.0,10.3,1.0,24.8,0.852778,0
4,F,64,WHITE,URGENT,Medicare,MARRIED,0,14.0,38.7,12.2,...,138.0,4.4,10.2,0.7,14.0,11.5,1.1,31.9,5.012500,0
5,M,43,ASIAN,EW EMER.,Other,MARRIED,1,12.0,37.6,12.5,...,141.0,4.3,8.9,0.9,12.0,10.2,0.9,29.9,72.651389,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1018,M,72,WHITE,OBSERVATION ADMIT,Medicare,MARRIED,1,24.0,33.8,10.7,...,144.0,4.0,9.0,1.2,24.0,13.6,1.2,28.1,23.528472,1
1019,M,69,WHITE,OBSERVATION ADMIT,Medicare,MARRIED,1,15.0,44.1,14.8,...,141.0,4.4,9.6,0.8,15.0,12.6,1.2,28.3,5.904167,1
1020,M,65,HISPANIC/LATINO - PUERTO RICAN,OBSERVATION ADMIT,Medicare,SINGLE,1,70.0,39.5,13.4,...,146.0,4.0,9.2,4.4,70.0,11.0,1.0,30.8,12.106944,1
1021,M,65,HISPANIC/LATINO - PUERTO RICAN,OBSERVATION ADMIT,Medicare,SINGLE,1,70.0,39.5,13.4,...,146.0,4.0,9.2,4.4,70.0,11.0,1.0,30.8,12.106944,1
